In [1]:
import pandas as pd

# Data Collection

In [2]:
# VEDL Stock Data
vedl_data = pd.read_csv('vedl_data.csv', parse_dates=['Date'], index_col=None)
print("VEDL Stock Data:")
print(vedl_data.head())

# Nifty Metal Stock Data
nifty_metal_data = pd.read_csv('nifty_metal_historical_data.csv', parse_dates=['Date'], index_col=None)
print("Nifty Metal Stock Data:")
print(nifty_metal_data.head())

# India's GDP/Infaltion Data
macroeconomic_data = pd.read_csv('india_gdp_inflation_data.csv', parse_dates=['Date'], index_col=None)
print("India's GDP/Infaltion Data:")
print(macroeconomic_data.head())

# News Sentiments Data
news_sentiment_data = pd.read_csv('news_with_sentiment_scores.csv', parse_dates=['date'], index_col=None)
news_sentiment_data.rename(columns={'date': 'Date'}, inplace=True)
news_sentiment_data['Date'] = news_sentiment_data['Date'].dt.date
news_sentiment_data['News Sentiment'] = news_sentiment_data[['title_sentiment_score', 'content_sentiment_score']].max(axis=1)
news_sentiment_data = news_sentiment_data[['Date','News Sentiment']]
print("News Sentiments Data:")
print(news_sentiment_data.head())

VEDL Stock Data:
   Unnamed: 0       Date        Open        High         Low       Close  \
0           0 2019-01-01  202.500000  202.850006  200.350006  202.100006   
1           1 2019-01-02  200.000000  200.000000  192.500000  193.149994   
2           2 2019-01-03  193.699997  195.000000  187.550003  187.949997   
3           3 2019-01-04  188.000000  194.449997  186.649994  192.899994   
4           4 2019-01-07  196.100006  198.649994  192.500000  193.449997   

   Adj Close    Volume  
0  88.260162   3947501  
1  84.351532  18935386  
2  82.080643  13968283  
3  84.242378  12883382  
4  84.482552  11020289  
Nifty Metal Stock Data:
        Date         Open         High          Low        Close    Adj Close  \
0 2019-01-01  3162.250000  3164.050049  3122.399902  3135.250000  3135.250000   
1 2019-01-02  3095.550049  3097.100098  3021.649902  3028.449951  3028.449951   
2 2019-01-03  3030.899902  3031.149902  2956.199951  2960.399902  2960.399902   
3 2019-01-04  2959.250000  3

# Data Preprocessing
##### Regularize the data for each date from 01-01-2019 to 24-10-2024, we can follow these steps:
######     Generate a date range covering all dates from 01-01-2019 to 24-10-2024.
######     Reindex each DataFrame to this date range.
######     Forward fill the missing values from the previous dates.
######     If there are any remaining missing values (e.g., at the start of the date range), fill these with the mean value of the column.

In [3]:
# Generate the complete date range
date_range = pd.date_range(start='2019-01-01', end='2024-10-24').date

# Function to regularize data
def regularize_data(df, date_range, columns_to_fill=None):
    df = df.set_index('Date').reindex(date_range).sort_index()
    if columns_to_fill is not None:
        df[columns_to_fill] = df[columns_to_fill].fillna(method='ffill') 
        df[columns_to_fill] = df[columns_to_fill].fillna(df[columns_to_fill].mean()) 
    else:
        df.fillna(method='ffill', inplace=True) 
        df.fillna(df.mean(), inplace=True) 
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    return df

In [4]:
# Apply the regularize_data function to each DataFrame
vedl_data_regularized = regularize_data(vedl_data, date_range)
nifty_metal_data_regularized = regularize_data(nifty_metal_data, date_range)
macroeconomic_data_regularized = regularize_data(macroeconomic_data, date_range)

C:\Users\prash\AppData\Local\Temp\ipykernel_38672\1130488957.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\prash\AppData\Local\Temp\ipykernel_38672\1130488957.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\prash\AppData\Local\Temp\ipykernel_38672\1130488957.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [5]:
# Create a DataFrame with the complete date range
complete_date_range = pd.DataFrame(date_range, columns=['Date'])

# Merge the sentiment data with the complete date range
news_sentiment_data_regularized = pd.merge(complete_date_range, news_sentiment_data, on='Date', how='left')

# Forward fill missing values
news_sentiment_data_regularized['News Sentiment'] = news_sentiment_data_regularized['News Sentiment'].fillna(method='ffill')

# Fill any remaining NaN values with the mean of the column
mean_sentiment = news_sentiment_data_regularized['News Sentiment'].mean()
news_sentiment_data_regularized['News Sentiment'] = news_sentiment_data_regularized['News Sentiment'].fillna(mean_sentiment)

C:\Users\prash\AppData\Local\Temp\ipykernel_38672\3795307701.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  news_sentiment_data_regularized['News Sentiment'] = news_sentiment_data_regularized['News Sentiment'].fillna(method='ffill')


In [6]:
# Verify the results
print("VEDL Stock Data:")
print(vedl_data_regularized.head())
print("\nNifty Metal Stock Data:")
print(nifty_metal_data_regularized.head())
print("\nIndia's GDP/Inflation Data:")
print(macroeconomic_data_regularized.head())
print("\nNews Sentiments Data:")
print(news_sentiment_data_regularized.head())

VEDL Stock Data:
         Date  Unnamed: 0        Open        High         Low       Close  \
0  2019-01-01         0.0  202.500000  202.850006  200.350006  202.100006   
1  2019-01-02         1.0  200.000000  200.000000  192.500000  193.149994   
2  2019-01-03         2.0  193.699997  195.000000  187.550003  187.949997   
3  2019-01-04         3.0  188.000000  194.449997  186.649994  192.899994   
4  2019-01-05         3.0  188.000000  194.449997  186.649994  192.899994   

   Adj Close      Volume  
0  88.260162   3947501.0  
1  84.351532  18935386.0  
2  82.080643  13968283.0  
3  84.242378  12883382.0  
4  84.242378  12883382.0  

Nifty Metal Stock Data:
         Date         Open         High          Low        Close  \
0  2019-01-01  3162.250000  3164.050049  3122.399902  3135.250000   
1  2019-01-02  3095.550049  3097.100098  3021.649902  3028.449951   
2  2019-01-03  3030.899902  3031.149902  2956.199951  2960.399902   
3  2019-01-04  2959.250000  3013.350098  2949.300049  300

In [7]:
# Merge all dataframes
merged_data = vedl_data_regularized.merge(nifty_metal_data_regularized, on='Date', suffixes=('_VEDL', '_NiftyMetal'))
merged_data = merged_data.merge(macroeconomic_data_regularized, on='Date')
merged_data = merged_data.merge(news_sentiment_data_regularized, on='Date')

# Drop any unnamed columns
merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('^Unnamed')]

# Verify the results
print("Merged Data:")
print(merged_data.head(10))

Merged Data:
         Date   Open_VEDL   High_VEDL    Low_VEDL  Close_VEDL  Adj Close_VEDL  \
0  2019-01-01  202.500000  202.850006  200.350006  202.100006       88.260162   
1  2019-01-02  200.000000  200.000000  192.500000  193.149994       84.351532   
2  2019-01-03  193.699997  195.000000  187.550003  187.949997       82.080643   
3  2019-01-04  188.000000  194.449997  186.649994  192.899994       84.242378   
4  2019-01-05  188.000000  194.449997  186.649994  192.899994       84.242378   
5  2019-01-06  188.000000  194.449997  186.649994  192.899994       84.242378   
6  2019-01-07  196.100006  198.649994  192.500000  193.449997       84.482552   
7  2019-01-08  194.500000  197.000000  191.899994  195.649994       85.443352   
8  2019-01-09  197.949997  198.000000  192.000000  194.750000       85.050293   
9  2019-01-10  195.000000  196.199997  193.449997  195.449997       85.356010   

   Volume_VEDL  Open_NiftyMetal  High_NiftyMetal  Low_NiftyMetal  \
0    3947501.0      3162.25

In [8]:
data = merged_data.copy()
data['Date'] = pd.to_datetime(data['Date'])

In [10]:
data.to_csv("final_merged_data.csv", index=False)
print("Merged data saved to data/final_merged_data.csv")

Merged data saved to data/final_merged_data.csv
